# 0. Imports

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import requests
import re
import time
from tqdm import tqdm
import numpy as np
import warnings
warnings.filterwarnings("ignore")
tqdm.pandas()


# 1. Leitura de bases

Os dados usados no desenvolvimento deste projeto surgiram a partir das bases presentes no link a seguir:

https://www.kaggle.com/datasets/natlee/myanimelist-comment-dataset

Osberve que a informação mais importante para nós são as que estão presentes na base `animeListGenres`

In [52]:
nRowsRead = 'None' # specify 'None' if want to read whole file

with open('animeReviewsOrderByTime.csv', 'r', encoding='utf-8') as f:
    headers = f.readline().replace('"','').replace('\n','').split(',')
    print(headers)
    print('The number of column: ', len(headers))
    dataFormat = dict()
    for header in headers:
        dataFormat[header] = list()

    for idx, line in enumerate(tqdm(f.readlines(), desc='Now parsing... ')):
        
        if idx == 67:
            yee = line
        
        if line != '':
            line = line.replace('\n','')
            indices = [i for i, x in enumerate(line) if x == ',']
            idxStart = 0
            for i in range(len(headers)):
                if i < len(headers) - 1:
                    dataFormat[headers[i]].append(line[idxStart + 1:indices[i] - 1])
                    idxStart = indices[i] + 1
                elif i == len(headers) - 1:
                    dataFormat[headers[i]].append(line[idxStart + 1:-1])
                else:
                    break
        if nRowsRead is not None and nRowsRead == idx + 1:
            print('We read only', nRowsRead, 'lines.')
            break
aval = pd.DataFrame(dataFormat)
aval.head()

['id', 'workId', 'reviewId', 'workName', 'postTime', 'episodesSeen', 'author', 'peopleFoundUseful', 'overallRating', 'storyRating', 'animationRating', 'soundRating', 'characterRating', 'enjoymentRating', 'review']
The number of column:  15


Now parsing... : 100%|██████████| 135206/135206 [00:14<00:00, 9347.88it/s] 


,id,workId,reviewId,workName,postTime,episodesSeen,author,peopleFoundUseful,overallRating,storyRating,animationRating,soundRating,characterRating,enjoymentRating,review
0,8121,1,1,Cowboy_Bebop,2006-11-07 18:34:00,26,Xinil,162,10,8,10,10,10,10,Cowboy Bebop is an episodic series. By episodi...
1,63480,856,9,Utawarerumono,2006-11-08 00:24:00,26,Crystal,218,8,8,7,7,9,8,Utawarerumono manages to be one of those harem...
2,8452,263,10,Hajime_no_Ippo,2006-11-08 14:39:00,76,Xinil,827,10,10,9,9,10,10,"First, let me say that I\\'m not a fan of boxi..."
3,66544,129,11,Gensoumaden_Saiyuuki,2006-11-09 05:22:00,50,Chelle,69,9,8,6,7,10,9,Saiyuki is one of those animes that just grabs...
4,55936,210,12,Ranma_½,2006-11-09 16:48:00,161,running_lemon,40,7,6,8,8,8,7,A comedy/romance based on the manga by Rumiko ...


In [3]:
#listinha serve para avaliar quais ids tivemoserro ao coletar a sinopse
listinha = []
def get_anime_synopsis(anime_id):
    time.sleep(1)
    url = f'https://myanimelist.net/anime/{anime_id}'
    response = requests.get(url)
    
    
    if response.status_code != 200:
        print(anime_id, "Parou aqui")
        listinha.append(anime_id)
        return "-" , "-"
    
    soup = BeautifulSoup(response.content, 'html.parser')
    synopsis_element = soup.find("p", itemprop="description")
    synopsis = synopsis_element.text.strip() if synopsis_element else 'Synopsis not found.'
    for image_tag in soup.find_all("img",itemprop="image"):
        image_link = image_tag['data-src']

    return synopsis , image_link

Como não temos a sinopse dos anims vamos coleta-las diretod a fonte, coletando de 100 em 100 e pausando por 10 min

In [3]:
# Carregando os dados
animes = pd.read_excel("Nase_animes_tratados.xlsx")

In [11]:
#Coletando sinopse dos animes
for i in tqdm(len(animes)):
    try:
        start = i * 100
        end = (i + 1) * 100 - 1
        result=pd.DataFrame()
        result[['synopsis', 'IMG']] = animes.loc[start:end, 'workId'].apply(get_anime_synopsis).apply(pd.Series)
        animes.loc[start:end, ['synopsis', 'IMG']] = result
    except Exception as e:
        print(f"An error occurred: {e}. Ignoring and moving on to the next chunk.")
        continue
    animes.to_excel("Nase_animes_tratados.xlsx", index = False)
    time.sleep(600)  # Sleep for 600 seconds (10 minutes) after each chunk

  0%|          | 0/8 [00:00<?, ?it/s]

6546 Parou aqui


 25%|██▌       | 2/8 [14:21<36:06, 361.08s/it]  

An error occurred: local variable 'image_link' referenced before assignment. Ignoring and moving on to the next chunk.


 38%|███▊      | 3/8 [14:32<16:47, 201.48s/it]

...

 62%|██████▎   | 5/8 [17:43<06:47, 135.93s/it]

An error occurred: local variable 'image_link' referenced before assignment. Ignoring and moving on to the next chunk.


 75%|███████▌  | 6/8 [17:49<03:03, 91.90s/it] 

An error occurred: local variable 'image_link' referenced before assignment. Ignoring and moving on to the next chunk.
39413 Parou aqui
39405 Parou aqui
39406 Parou aqui
39536 Parou aqui
39502 Parou aqui
39501 Parou aqui
39540 Parou aqui
39315 Parou aqui
39564 Parou aqui
39517 Parou aqui
39310 Parou aqui
39342 Parou aqui
39344 Parou aqui
39343 Parou aqui
39384 Parou aqui
39550 Parou aqui
39322 Parou aqui
39369 Parou aqui
39370 Parou aqui
39407 Parou aqui
39307 Parou aqui
39431 Parou aqui
39621 Parou aqui
39330 Parou aqui
39376 Parou aqui
39720 Parou aqui
39375 Parou aqui
39541 Parou aqui
39546 Parou aqui
39374 Parou aqui
39403 Parou aqui
39371 Parou aqui
39372 Parou aqui
39306 Parou aqui
39566 Parou aqui
39488 Parou aqui
39333 Parou aqui
39360 Parou aqui
39457 Parou aqui
39544 Parou aqui
39507 Parou aqui
39568 Parou aqui
39513 Parou aqui
39358 Parou aqui
39361 Parou aqui
39359 Parou aqui
39335 Parou aqui
39503 Parou aqui
39453 Parou aqui
39352 Parou aqui
39581 Parou aqui
39451 Parou aq

100%|██████████| 8/8 [40:32<00:00, 304.07s/it]


# 2. Tratamento dos dados

In [5]:
animes = animes[(animes["IMG"] !="-") & (animes["synopsis"] !="-")]
# Removendo duplicatas
animes.drop_duplicates(inplace=True)
aval.drop_duplicates(inplace=True)

# Tratando valores faltantes
animes.fillna("", inplace=True)
aval.fillna("", inplace=True)

# Instanciando o stemmer and the lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


In [40]:
import nltk
from nltk.corpus import stopwords

# Baixe as stopwords em português (se ainda não tiver baixado)
nltk.download('stopwords')

# Carregue as stopwords em português
remover_palavras = stopwords.words('english')#stopwords.words('portuguese')

from nltk.stem import RSLPStemmer
import spacy

# Baixe os recursos necessários (se ainda não tiver baixado)
nltk.download('rslp')
nltk.download('wordnet')

# Crie uma instância do stemmer RSLP
stemmer = RSLPStemmer()

# Crie uma instância do lematizador para o português
nlp = spacy.load('en_core_web_sm')#spacy.load('pt_core_news_sm')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:
# Function to remove HTML tags
def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Function to remove URLs
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

# Function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Function to preprocess the text
def preprocess(text):
    try:
        # Removing HTML tags
        text = remove_html(text)
    except:
        pass
        # Removing URLs
    try:
        text = remove_urls(text)
    except:
        pass
        # Removing emojis
    try:
        text = remove_emojis(text)
    except:
        pass
        # Converting to lowercase
    try:
        text = text.lower()
    except:
        pass
        # Removing special characters
    try:
        text = re.sub(r'\W', ' ', text)
    except:
        pass
        # Removing extra spaces
    try:
        text = re.sub(r'\s+', ' ', text, flags=re.I)
    except:
        pass
        # Removing stopwords, applying lemmatization and then stemming
    try:
        #text = ' '.join([stemmer.stem(lemmatizer.lemmatize(word)) for word in text.split() if word not in remover_palavras])
        text = ' '.join([stemmer.stem(token.lemma_) for token in nlp(text) if token.lemma_ not in remover_palavras])
    except:
        pass
    
    return text


In [31]:
# Applying the preprocess function to the synopsis and user comments
animes['synopsis tratada'] = animes['synopsis'].progress_apply(preprocess)


 32%|███▏      | 43044/135206 [23:56<51:15, 29.97it/s]  


IndexError: string index out of range

In [43]:
aval['review tratada'] = aval['review'].progress_apply(preprocess)

100%|██████████| 135206/135206 [2:08:57<00:00, 17.47it/s]  


In [44]:
animes['synopsis tratada final'] = animes['synopsis tratada'].progress_apply(preprocess)

100%|██████████| 14728/14728 [01:43<00:00, 142.31it/s] 


In [48]:
animes.reset_index(inplace= True, drop = True)

In [30]:
#Existem alguns nomes em branco
animes["engName"].value_counts()

\N                                                         6265
Cyborg 009                                                    4
Outlaw Star                                                   3
The Heroic Legend of Arslan                                   3
JoJo's Bizarre Adventure                                      3
                                                           ... 
Astro Boy (1980)                                              1
Armored Trooper Votoms: The Last Red Shoulder                 1
Shuffle!                                                      1
Spaceketeers                                                  1
UzaMaid! My Maid Is Still Seriously Way Too Annoying...       1
Name: engName, Length: 5162, dtype: int64

In [34]:
#atualizando nome dos animes
def atualizar_nome(row):
    if row["engName"] != "\\N":
        return row["engName"]
    elif row["synonymsName"] != "\\N":
        return row["synonymsName"]
    else:
        return row["jpName"]
animes["name"] = animes.apply(lambda row: atualizar_nome(row), axis = 1)


In [35]:
#Note que reduzimos os sem nome para 13, vamos mantelos
animes["name"].value_counts()

Minna no Uta                                                                          67
\N                                                                                    13
Irodorimidori                                                                          7
Wakate Animator Ikusei Project, 2018 Young Animator Training Project, Anime Tamago     4
Spooky Kitaro                                                                          4
                                                                                      ..
BROTHERS CONFLICT OVA                                                                  1
Arjuna                                                                                 1
Children Record                                                                        1
CLAMP IN WONDERLAND                                                                    1
UzaMaid! My Maid Is Still Seriously Way Too Annoying...                                1
Name: name, Length: 1

In [36]:
animes = animes[-animes["synopsis tratada final"].isna()].reset_index(drop=True)
animes.to_excel("base_tratada_final_animes.xlsx",index = False)

In [58]:
aval.to_csv("reviews_tratada.csv", index = False)

# 3. Similadirade

In [7]:
aval = pd.read_csv("reviews_tratada.csv")
animes = pd.read_excel("base_tratada_final_animes.xlsx")

In [37]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Vectorize the preprocessed anime descriptions
vectorizer = TfidfVectorizer()
vectorized_descriptions = vectorizer.fit_transform(animes['synopsis tratada final'])

# Calculate similarity scores
similarity_matrix = cosine_similarity(vectorized_descriptions)

# Function to retrieve top similar animes for a given anime
def get_top_similar_animes(anime_index, similarity_matrix, top_k=5):
    anime_scores = similarity_matrix[anime_index]  # Get similarity scores for the given anime
    top_indices = anime_scores.argsort()[-top_k-1:-1][::-1]  # Get top similar anime indices (excluding the given anime itself)
    return animes.iloc[top_indices]['workId'].values.tolist()




In [38]:
np.save("vector", similarity_matrix)   